In [1]:
import json
import spacy
from spacy import displacy
import random
from nltk.corpus import stopwords

In [2]:
stop_words_english = set(stopwords.words('english'))


In [3]:
with open("/home/leon/Desktop/market/data/all_data2.json", "r") as f:
    data = json.load(f)

In [4]:
data["reviews"][1]["review_text"]

'obsessed!!!'

In [5]:
def extract_words(text, spacy_object, stop_words):
    keywords = []
    doc_object = spacy_object(text)
    for token in doc_object:
        if token.pos_ == "ADJ":
            if token.dep_ in ["amod", "conj"]:
                keywords += [token]

        if token.dep_ == "nsubj":
            for child in token.head.children:
                if child.pos_ == "ADJ":
                    keywords += [child]

        if token.dep_ in ["dobj", "acomp"]:
            keywords += [token]
            
        if token.dep_ == "conj" and token.head.pos_ == "VERB":
            keywords += [token]

        if token.dep_ == "advmod" and token.pos_ == "ADV" and token.head.pos_ == "ADJ":
            keywords += [token]
        
        if len([c for c in token.children]) > 3 and token.text not in stop_words:
            keywords += [token]
            
    if len(keywords) == 0:
        keywords = list(doc_object)

    if len(doc_object) == 0 :
        keywords = list(spacy_object("and"))
        
    return(list(set(keywords)))

In [6]:
nlp = spacy.load('en_core_web_md')

In [7]:
index = random.randint(1,5000)
ex = data["reviews"][index]["review_text"]
doc = nlp(ex)
ex

"Best face mask I've ever tried"

In [8]:
extract_words(ex, nlp, stop_words_english)




[Best]

In [9]:
displacy.render(doc, style='dep', jupyter=True)

In [10]:
import numpy as np
import pandas as pd

In [11]:
len(data["reviews"])

5570

In [12]:

def get_vector(text, spacy_object, stop_words):
    
    words = extract_words(text, spacy_object, stop_words)
    table = np.array([x.vector for x in words])
    review_vector = np.amax(table, 0)
    
    return (review_vector)


In [13]:
vector_data = []

In [14]:
for x in np.arange(len(data["reviews"])):
    if x % 100 == 0:
        print(x)
    
    try:
        review_vector = get_vector(data["reviews"][x]["review_text"], nlp, stop_words_english)
        vector_data += [review_vector]
    except:
        print(data["reviews"][x]["review_text"])
        break
    
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500


In [15]:
any([len(x) != 300 for x in vector_data])

False

In [39]:
vector_data[0]

array([-3.1342e-01,  3.7267e-01, -4.1600e-01,  1.8519e-01,  8.4357e-02,
        5.3412e-02,  2.5342e-01, -4.4226e-01,  8.0791e-02,  2.4691e+00,
       -1.7312e-02, -2.1510e-01, -3.5174e-02, -1.0650e-01, -3.3082e-01,
        3.9420e-02, -2.6468e-01,  1.3285e+00,  1.9279e-01,  5.3120e-03,
       -1.2395e-01, -2.8884e-01, -5.2645e-03, -4.2899e-01,  3.4108e-02,
        4.3833e-01,  1.8667e-01, -1.5704e-02,  2.5948e-01,  3.1607e-02,
       -1.7761e-01, -9.6713e-02,  1.7473e-01,  3.3382e-01,  2.4716e-01,
       -1.1465e-01,  9.9883e-02, -4.1078e-02, -2.9376e-01, -1.0925e-01,
        2.2638e-01,  5.4675e-02,  3.0214e-02, -1.0756e-01,  4.2631e-01,
        1.4436e-01, -1.2029e-01,  7.3671e-02,  2.6864e-01, -4.9410e-02,
       -3.6164e-01,  4.5891e-01, -5.0007e-02,  8.2917e-02,  3.1291e-01,
        6.2817e-01,  6.4266e-02,  3.7151e-01,  1.0614e-01,  2.1074e-01,
        3.2074e-01, -3.8285e-01,  1.3460e-02,  5.1735e-02, -8.9817e-02,
       -2.5087e-01, -5.7980e-02,  5.1106e-01,  2.6085e-01,  1.89

array([[-0.31342 ,  0.37267 , -0.416   , ...,  0.15438 ,  0.12621 ,
         0.49521 ],
       [ 0.06452 ,  0.33531 ,  0.2186  , ..., -0.086928,  0.095215,
         0.16232 ],
       [ 0.14147 ,  0.17264 , -0.35865 , ..., -0.1464  , -0.2718  ,
         0.49267 ],
       ...,
       [ 0.053618,  0.35653 ,  0.063868, ...,  0.03446 ,  0.37132 ,
         0.68995 ],
       [ 0.32257 ,  0.24589 ,  0.23053 , ...,  0.49122 ,  0.18915 ,
         0.65465 ],
       [ 0.044704,  0.65344 ,  0.063868, ...,  0.1727  ,  0.39303 ,
         0.49393 ]], dtype=float32)

In [43]:
dataframe = pd.DataFrame(np.array(vector_data),columns=np.arange(len(vector_data[0])))

In [44]:
dataframe

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.313420,0.37267,-0.416000,0.185190,0.084357,0.053412,0.253420,-0.442260,0.080791,2.4691,...,-0.124850,0.365280,0.638450,0.185980,0.515560,0.297570,0.341100,0.154380,0.126210,0.49521
1,0.064520,0.33531,0.218600,-0.067463,-0.055470,0.124550,0.341280,-0.163340,-0.007531,1.8152,...,-0.112000,-0.017258,0.217560,0.020728,0.283970,0.487980,0.151950,-0.086928,0.095215,0.16232
2,0.141470,0.17264,-0.358650,-0.454900,0.266290,0.198000,0.110480,0.028269,0.342530,1.8359,...,-0.323970,-0.156280,-0.203740,-0.210670,0.286280,0.036254,0.092874,-0.146400,-0.271800,0.49267
3,0.212360,0.50200,0.185930,0.185190,0.307210,0.102040,0.771570,-0.071709,0.276370,2.6751,...,0.262680,0.501520,0.604600,0.473650,0.515560,0.528160,0.334740,0.142620,0.639790,0.47315
4,0.141470,0.60963,-0.019271,0.045865,0.266290,0.198000,0.301630,0.028269,0.342530,2.5797,...,-0.003889,0.129150,-0.047795,-0.015365,0.286280,0.247890,0.409600,0.068683,0.283640,0.49267
5,0.001363,0.35653,0.071775,-0.053600,0.003140,0.223390,0.452380,-0.205380,-0.061855,2.8955,...,-0.150440,0.465370,0.330280,0.281810,0.447690,0.420790,0.055763,0.058612,0.078259,0.22398
6,0.080680,0.45638,0.094474,0.097579,0.102810,0.052494,0.489250,0.315730,0.396300,2.7681,...,0.338400,0.468520,0.191080,0.445250,0.187040,-0.146370,0.033012,0.235630,0.553880,0.27566
7,0.141470,0.50881,0.063868,0.292920,0.266290,0.221360,0.429190,0.028269,0.342530,2.6236,...,-0.039495,0.501520,0.248710,0.141500,0.442740,0.225870,0.219620,0.107380,0.281160,0.87864
8,0.247470,0.33531,0.218600,-0.208300,0.323120,0.252280,0.172360,0.243500,0.264620,2.2187,...,0.535000,0.152320,0.299950,0.450870,0.283970,0.360650,0.571500,0.388420,0.231630,0.30791
9,0.030971,0.46795,0.058068,0.002916,0.125530,0.245610,0.082157,0.343810,0.098002,2.6992,...,-0.229580,0.277910,0.020619,-0.018093,0.218150,-0.097673,0.455330,0.147700,0.229350,0.22752


In [45]:
dataframe.to_csv("/home/leon/Desktop/market/data/vector_data.csv", header=True, index=True)

In [46]:
dataframe = pd.read_csv("/home/leon/Desktop/market/data/vector_data.csv", header=0, index_col=0)

In [47]:
ratings = [float(data["reviews"][x]["review_rating"])*30 for x in np.arange(len(data["reviews"]))]



In [48]:
len(data["reviews"])

5570

In [49]:
ratings = pd.DataFrame(ratings)
ratings.shape

(5570, 1)

In [50]:
master_table = pd.concat([ratings, dataframe], axis=1)

In [51]:
master_table

,0,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,150.0,-0.313420,0.37267,-0.416000,0.185190,0.084357,0.053412,0.253420,-0.442260,0.080791,...,-0.124850,0.365280,0.638450,0.185980,0.515560,0.297570,0.341100,0.154380,0.126210,0.49521
1,150.0,0.064520,0.33531,0.218600,-0.067463,-0.055470,0.124550,0.341280,-0.163340,-0.007532,...,-0.112000,-0.017258,0.217560,0.020728,0.283970,0.487980,0.151950,-0.086928,0.095215,0.16232
2,150.0,0.141470,0.17264,-0.358650,-0.454900,0.266290,0.198000,0.110480,0.028269,0.342530,...,-0.323970,-0.156280,-0.203740,-0.210670,0.286280,0.036254,0.092874,-0.146400,-0.271800,0.49267
3,150.0,0.212360,0.50200,0.185930,0.185190,0.307210,0.102040,0.771570,-0.071709,0.276370,...,0.262680,0.501520,0.604600,0.473650,0.515560,0.528160,0.334740,0.142620,0.639790,0.47315
4,150.0,0.141470,0.60963,-0.019271,0.045865,0.266290,0.198000,0.301630,0.028269,0.342530,...,-0.003889,0.129150,-0.047795,-0.015365,0.286280,0.247890,0.409600,0.068683,0.283640,0.49267
5,30.0,0.001363,0.35653,0.071775,-0.053600,0.003140,0.223390,0.452380,-0.205380,-0.061855,...,-0.150440,0.465370,0.330280,0.281810,0.447690,0.420790,0.055763,0.058612,0.078259,0.22398
6,150.0,0.080680,0.45638,0.094474,0.097579,0.102810,0.052494,0.489250,0.315730,0.396300,...,0.338400,0.468520,0.191080,0.445250,0.187040,-0.146370,0.033012,0.235630,0.553880,0.27566
7,150.0,0.141470,0.50881,0.063868,0.292920,0.266290,0.221360,0.429190,0.028269,0.342530,...,-0.039495,0.501520,0.248710,0.141500,0.442740,0.225870,0.219620,0.107380,0.281160,0.87864
8,150.0,0.247470,0.33531,0.218600,-0.208300,0.323120,0.252280,0.172360,0.243500,0.264620,...,0.535000,0.152320,0.299950,0.450870,0.283970,0.360650,0.571500,0.388420,0.231630,0.30791
9,150.0,0.030971,0.46795,0.058068,0.002916,0.125530,0.245610,0.082157,0.343810,0.098002,...,-0.229580,0.277910,0.020619,-0.018093,0.218150,-0.097673,0.455330,0.147700,0.229350,0.22752


In [52]:
master_table.columns = [x for x in np.arange(301)]
master_table.to_csv("/home/leon/Desktop/market/data/vector_data_ratings.csv", header=True, index=True)